### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [ ]:
# Calculate the total number of schools; find the total number of unique schools
unique_schools = school_data_complete["school_name"].unique()
school_count = len(unique_schools)

# Calculate the total number of students
total_students = school_data_complete["student_name"].count()

# Calculate the total budget; find unique budget values and add together
unique_budgets = school_data_complete["budget"].unique()
total_budget = sum(unique_budgets)

# Calculate the average math score
av_math = school_data_complete["math_score"].mean()

# Calculate the average reading score
av_read = school_data_complete["reading_score"].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
# Create dataframe with math scores > 70 and count number of students
math_pass_df = school_data_complete.loc[school_data_complete["math_score"] >= 70, [
    "school_name", "student_name", "math_score", "reading_score"]]
math_pass_rate = math_pass_df["student_name"].count()/total_students*100

# Calculate the percentage of students with a passing reading score (70 or greater)
# Create dataframe with reading scores > 70 and count number of students
read_pass_df = school_data_complete.loc[school_data_complete["reading_score"] >= 70, [
    "school_name", "student_name", "reading_score"]]
read_pass_rate = read_pass_df["student_name"].count()/total_students*100

# Calculate the percentage of students who passed math and reading (% Overall Passing)
# Create dataframe with math and reading scores > 70 and count number of students
both_pass_df = math_pass_df.loc[math_pass_df["reading_score"] >= 70, ["school_name", "student_name", 
                                                                      "math_score", "reading_score"]]
both_pass_rate = both_pass_df["student_name"].count()/total_students*100

# Create a dataframe to hold the above results
district_summary_df = pd.DataFrame({"Total Schools": [school_count],
                                    "Total Students": [total_students],
                                    "Total Budget": [total_budget],
                                    "Average Math Score": [av_math],
                                    "Average Reading Score": [av_read],
                                    "% Passing Math": [math_pass_rate],
                                    "% Passing Reading": [read_pass_rate],
                                    "% Overall Passing": [both_pass_rate]})

# Optional: give the displayed data cleaner formatting
district_summary_df["Total Budget"] = district_summary_df["Total Budget"].map("${:,.2f}".format)
district_summary_df["Total Students"] = district_summary_df["Total Students"].map("{:,}".format)
district_summary_df

## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [ ]:
# Group by School Name and create data frame for averages and counts
schools_group = school_data_complete.groupby("school_name")
ave_df = pd.DataFrame(schools_group[["math_score", "reading_score", "budget"]].mean())
count_df = pd.DataFrame(schools_group[["student_name"]].count())

# Merge created data frames based on school_name
school_merge_df = pd.merge(count_df, ave_df, on="school_name")
school_merge_df=school_merge_df.rename(columns={"student_name":"Total Students", 
                                                "math_score":"Average Math Score",
                                                "reading_score":"Average Reading Score",
                                                "budget": "Total School Budget"})

# Calculate per student budget
school_merge_df["Per Student Budget"]=school_merge_df["Total School Budget"]/school_merge_df["Total Students"]

# Calculate number of students who passed using test score data frames
# Calculate total number of students who passed math by school
math_group = math_pass_df.groupby("school_name")
math_group_pass_df = pd.DataFrame(math_group[["student_name"]].count())
school_mergem_df = pd.merge(school_merge_df, math_group_pass_df, on="school_name")
school_mergem_df=school_mergem_df.rename(columns={"student_name":"Students Who Passed Math"})

# Calculate total number of students who passed reading by school
read_group = read_pass_df.groupby("school_name")
read_group_pass_df = pd.DataFrame(read_group[["student_name"]].count())
school_merger_df = pd.merge(school_mergem_df, read_group_pass_df, on="school_name")
school_merger_df=school_merger_df.rename(columns={"student_name":"Students Who Passed Reading"})

# Calculate total number of students who passed both math & reading by school
both_group = both_pass_df.groupby("school_name")
both_group_pass_df = pd.DataFrame(both_group[["student_name"]].count())
school_mergeb_df = pd.merge(school_merger_df, both_group_pass_df, on="school_name")
school_mergeb_df=school_mergeb_df.rename(columns={"student_name":"Students Who Passed Both"})

# Calculate percentage of students who passed
school_mergeb_df["% Passing Math"] = school_mergeb_df["Students Who Passed Math"]/school_mergeb_df["Total Students"]*100
school_mergeb_df["% Passing Reading"] = school_mergeb_df["Students Who Passed Reading"]/school_mergeb_df["Total Students"]*100
school_mergeb_df["% Overall Passing"] = school_mergeb_df["Students Who Passed Both"]/school_mergeb_df["Total Students"]*100

# Add School Type to data frame with merge and create School Type DF
school_type_df = school_data[["school_name", "type"]]
school_mergef_df = pd.merge(school_mergeb_df, school_type_df, on="school_name", how="left")
school_mergef_df=school_mergef_df.rename(columns={"type":"School Type"})

# Create final data frame and make it pretty
final_school_df = school_mergef_df[["school_name", "School Type", "Total Students", "Total School Budget", "Per Student Budget",
                                    "Average Math Score", "Average Reading Score", "% Passing Math", 
                                    "% Passing Reading", "% Overall Passing"]]

tidy_final_school_df = final_school_df.set_index("school_name")
tidy_final_school_df.index.name = None
tidy_final_school_df["Total School Budget"] = tidy_final_school_df["Total School Budget"].map("${:,.2f}".format)
tidy_final_school_df["Per Student Budget"] = tidy_final_school_df["Per Student Budget"].map("${:,.2f}".format)

tidy_final_school_df

## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [ ]:
# Sort final dataframe to find top 5 performing schools
best_school_df = tidy_final_school_df.sort_values("% Overall Passing", ascending=False)
best_school_df.iloc[0:5,:]

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [ ]:
# Sort final dataframe to find top 5 performing schools
worst_school_df = tidy_final_school_df.sort_values("% Overall Passing")
worst_school_df.iloc[0:5,:]


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
# Isolate ninth_grade math scores and create series
ninth_math = school_data_complete.loc[school_data_complete["grade"] == "9th", :]
ninth_math_scores = pd.Series(ninth_math["math_score"])
ninth_math_scores = ninth_math_scores.set_axis(ninth_math["school_name"])
ninth_math_group = ninth_math_scores.groupby("school_name").mean()

# Isolate tenth_grade math scores and create series
tenth_math = school_data_complete.loc[school_data_complete["grade"] == "10th", :]
tenth_math_scores = pd.Series(tenth_math["math_score"])
tenth_math_scores = tenth_math_scores.set_axis(tenth_math["school_name"])
tenth_math_group = tenth_math_scores.groupby("school_name").mean()

# Isolate eleventh_grade math scores and create series
eleventh_math = school_data_complete.loc[school_data_complete["grade"] == "11th", :]
eleventh_math_scores = pd.Series(eleventh_math["math_score"])
eleventh_math_scores = eleventh_math_scores.set_axis(eleventh_math["school_name"])
eleventh_math_group = eleventh_math_scores.groupby("school_name").mean()

# Isolate ninth_grade math scores and create series
twelfth_math = school_data_complete.loc[school_data_complete["grade"] == "12th", :]
twelfth_math_scores = pd.Series(twelfth_math["math_score"])
twelfth_math_scores = twelfth_math_scores.set_axis(twelfth_math["school_name"])
twelfth_math_group = twelfth_math_scores.groupby("school_name").mean()

# Create Data Frame with all averages by grade by merging
math_merge1_df = pd.merge(ninth_math_group, tenth_math_group, on="school_name")
math_merge1_df = math_merge1_df.rename(columns={"math_score_x":"9th",
                                                "math_score_y":"10th"})

math_merge2_df = pd.merge(math_merge1_df, eleventh_math_group, on="school_name")
math_merge2_df = math_merge2_df.rename(columns={"math_score":"11th"})

math_merge3_df = pd.merge(math_merge2_df, twelfth_math_group, on="school_name")
math_merge3_df = math_merge3_df.rename(columns={"math_score":"12th"})

math_merge3_df.index.name = None
math_merge3_df

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:

# Isolate ninth_grade reading scores and create series
ninth_reading = school_data_complete.loc[school_data_complete["grade"] == "9th", :]
ninth_reading_scores = pd.Series(ninth_reading["reading_score"])
ninth_reading_scores = ninth_reading_scores.set_axis(ninth_reading["school_name"])
ninth_reading_group = ninth_reading_scores.groupby("school_name").mean()

# Isolate tenth_grade reading scores and create series
tenth_reading = school_data_complete.loc[school_data_complete["grade"] == "10th", :]
tenth_reading_scores = pd.Series(tenth_reading["reading_score"])
tenth_reading_scores = tenth_reading_scores.set_axis(tenth_reading["school_name"])
tenth_reading_group = tenth_reading_scores.groupby("school_name").mean()

# Isolate eleventh_grade reading scores and create series
eleventh_reading = school_data_complete.loc[school_data_complete["grade"] == "11th", :]
eleventh_reading_scores = pd.Series(eleventh_reading["reading_score"])
eleventh_reading_scores = eleventh_reading_scores.set_axis(eleventh_reading["school_name"])
eleventh_reading_group = eleventh_reading_scores.groupby("school_name").mean()

# Isolate ninth_grade reading scores and create series
twelfth_reading = school_data_complete.loc[school_data_complete["grade"] == "12th", :]
twelfth_reading_scores = pd.Series(twelfth_reading["reading_score"])
twelfth_reading_scores = twelfth_reading_scores.set_axis(twelfth_reading["school_name"])
twelfth_reading_group = twelfth_reading_scores.groupby("school_name").mean()

# Create Data Frame with all averages by grade by merging
reading_merge1_df = pd.merge(ninth_reading_group, tenth_reading_group, on="school_name")
reading_merge1_df = reading_merge1_df.rename(columns={"reading_score_x":"9th",
                                                "reading_score_y":"10th"})

reading_merge2_df = pd.merge(reading_merge1_df, eleventh_reading_group, on="school_name")
reading_merge2_df = reading_merge2_df.rename(columns={"reading_score":"11th"})

reading_merge3_df = pd.merge(reading_merge2_df, twelfth_reading_group, on="school_name")
reading_merge3_df = reading_merge3_df.rename(columns={"reading_score":"12th"})

reading_merge3_df.index.name = None
reading_merge3_df

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Create the Spending Range bins in which Data will be held 
spend_bins = [0, 585, 630, 645, 675]

# Create the names for the four bins
spend_bin_names = ["<$584", "$585-629", "$630-644", "$645-675"]

# Add bins to final summary data frame
final_school_df["Spending Ranges (Per Student)"] = pd.cut(final_school_df["Per Student Budget"], spend_bins, 
                                                          labels=spend_bin_names)

# Sort values by Spending Range
scores_by_spend = final_school_df.groupby("Spending Ranges (Per Student)")
scores_by_spend_df = pd.DataFrame(scores_by_spend[["Average Math Score", "Average Reading Score", "% Passing Math", 
                                                 "% Passing Reading", "% Overall Passing"]].mean())
scores_by_spend_df

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Create the School Size bins in which Data will be held 
size_bins = [0, 1000, 2000, 5000]

# Create the names for the three bins
size_bin_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Add school size bins to final summary data frame
final_school_df["School Size"] = pd.cut(final_school_df["Total Students"], size_bins, 
                                                          labels=size_bin_names)

# Sort values by School Size
scores_by_size = final_school_df.groupby("School Size")
scores_by_size_df = pd.DataFrame(scores_by_size[["Average Math Score", "Average Reading Score", "% Passing Math", 
                                                 "% Passing Reading", "% Overall Passing"]].mean())
scores_by_size_df

## Scores by School Type

* Perform the same operations as above, based on school type

In [ ]:
# Sort values by School Type
scores_by_type = final_school_df.groupby("School Type")
scores_by_type_df = pd.DataFrame(scores_by_type[["Average Math Score", "Average Reading Score", "% Passing Math", 
                                                 "% Passing Reading", "% Overall Passing"]].mean())
scores_by_type_df